In [ ]:
import pandas as pd
import joblib
import json
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

# Load models
onehot_encoders = joblib.load('onehot_encoders_loan.joblib')
xgb_clf = joblib.load('xgb_classifier_loan.joblib')

# Mapping back from prediction
risk_reverse_mapping = {0: 'low', 1: 'medium', 2: 'high'}

def preprocess_input(json_data):
    # Convert JSON to DataFrame
    input_df = pd.DataFrame([json_data])

    # Drop irrelevant columns
    input_df = input_df.drop(["city", "district", "customer_id"], axis=1, errors='ignore')

    # Separate categorical and numerical
    categorical_cols = [col for col in onehot_encoders.keys()]
    numerical_cols = input_df.select_dtypes(include=['number']).columns.tolist()

    # Encode categorical
    processed_df = pd.DataFrame(index=input_df.index)
    for col in categorical_cols:
        encoder = onehot_encoders[col]
        encoded_feature = encoder.transform(input_df[[col]])
        feature_names = encoder.get_feature_names_out([col])
        encoded_df = pd.DataFrame(encoded_feature, columns=feature_names, index=input_df.index)
        processed_df = pd.concat([processed_df, encoded_df], axis=1)

    # Append numerical
    numerical_df = input_df[numerical_cols]
    final_input_df = pd.concat([processed_df, numerical_df], axis=1)

    return final_input_df

def predict_risk(json_data):
    processed_input = preprocess_input(json_data)
    pred = xgb_clf.predict(processed_input)[0]
    return risk_reverse_mapping[pred]
